# Installieren des Python Modules radproc

**Vorraussetzung ist Python 2 und ArcGIS 64 Bit Extension für Python.**

In [ ]:
!pip install radproc

# Modul impotieren 

In [ ]:
import radproc as rp

# Die Daten von ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/radolan/historical/bin/2018/ in einem Ordner Speichern (ftp_cdc), einen Ordner erstellen wo radproc die entpackten Daten lagern kann

In [ ]:
RW_original = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\ftp_cdc"
RW_unzipped = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\RW_unzipped"

rp.unzip_RW_binaries(zipFolder=RW_original, outFolder=RW_unzipped)

# Radproc entpackt nun die Daten in RW_unzipped/Jahr/Monat. Jetzt die Daten impotieren und als HDF5 expotieren, Wenn gewünscht kann eine Shape-Datei als Boundary dienen 

**create an ID-raster for Germany in ArcGIS, called idras_ger,
if you specified a Shapefile or Feature-Class as clipFeature: Clip the german ID-raster to the extent of the clipFeature and create a second ID-raster called idras,
import all RADOLAN binary files in a directory tree,
select the data for your study area based on the generated ID-raster,
convert the selected data into monthly pandas DataFrames and
store all DataFrames in the specified HDF5 file.**


In [ ]:
RW_unzipped = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\RW_unzipped"
outHDF = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi\out\aoi.h5"
studyArea = r"C:\Users\USER\Desktop\Masterarbeit\DATA\boundarys\aoi_land\aoi_land.shp"

rp.create_idraster_and_process_radolan_data(inFolder=RW_unzipped,clipFeature=studyArea, HDFFile=outHDF, complevel=9) #clipFeature=studyArea

# Niederschlagssummen 

In [ ]:
HDFFile = outHDF

**Lade Daten von HDF5 und Berechne die järhlichen Niederschlagssumen**

In [ ]:
month=0
annualSum = rp.hdf5_to_years("C:/Users/USER/Desktop/Masterarbeit/DATA/RAD/aoi_land/out_HDF/aoi_land.h5", 2016)
# Display the first five rows of the new DataFrame
annualSum.head()

**Monatliche Niederschlagssumme**

In [ ]:
#test = rp.core.load_months_from_hdf5(r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi_land\out_HDF\aoi_land.h5",2017)
monthlySum = rp.hdf5_to_months(r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi_land\out_HDF\aoi_land.h5",2017, 2018)
# Display the first five rows of the new DataFrame
monthlySum.head()

**Tägliche Niederschlagssumme**

In [ ]:
dailySum = rp.hdf5_to_days(r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi\out\aoi.h5", year_start=2014, year_end=2018)
dailySum.head(2)
print(dailySum.index)
print(dailySum.columns)

#hourlySum = rp.core.hdf5_to_days(HDF, 2016, 2018)
#hourlySum.head()


# Export to ArcGIS Geodatabase 

**Export Jährliche Niederschlagssumme**

In [ ]:
idRaster = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi\out\idras"
outGDBPath = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi"
GDBName = "aoi_pp_2014_2018.gdb"
statistics = ["mean", "max","min","range"]

rp.export_dfrows_to_gdb(dataDF=hourlySum, idRaster=idRaster, outGDBPath=outGDBPath, GDBName=GDBName, statistics=statistics)

**Export Monatliche Niederschlagssumme**

In [ ]:
GDBName = "Months_16_18.gdb"
statistics = ["mean", "max", "min", "range"]

rp.export_dfrows_to_gdb(dataDF=monthlySum, idRaster=idRaster, outGDBPath=outGDBPath, GDBName=GDBName, statistics=statistics)

In [ ]:
#Export Daily PP to GeoDataBase
idRaster = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi\out\idras"
outGDBPath = r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi"
GDBName = "aoi_pp_2014_2018.gdb"
statistics = ["mean", "max","min","range"]

rp.export_dfrows_to_gdb(dataDF=hourlySum, idRaster=idRaster, outGDBPath=outGDBPath, GDBName=GDBName, statistics=statistics)

# Heavy Rainfall Analyse 

In [ ]:
df = rp.hdf5_to_days(r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi\out\aoi.h5", year_start=2014, year_end=2018)
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

#Existing Data
index_dates =  pd.date_range(pd.datetime.today(), periods=10).tolist()
df = pd.DataFrame({'486335':[0,0,16,0,0,0,2,1,8,2],'486336':[2,1,8,0,11,16,0,1,6,8],'486337':[22,1,22,0,0,0,5,3,6,1]},index=index_dates)
columns = df.columns 
counter_columns = 0
print(df.head(5))
iteration = -1 #Iterations Steps
counter = 10 #10 precipitation values per column
duration = 0 #days with no or less than pp_max_1 rain 
count = False

index_list = df.index #Index for updating df / Integear
period_range = 0  #Amount of days after Event without much rain Integear
period_amount = 0 #Amount of PP in dry days except event Integear
event_amount = 0.0  #Amount of heavy rainfall on the event date Float
pp = 0 #actual precipitation
pp_sum = 0.0 #mm
pp_min = 15.0 #mm min pp for start to count dry days until duration_min_after
pp_max_1 = 0.11 #max pp for 1 day while counting dry days
dry_days = 0 #dry days after event

for x in df:
    for y in df[x]:
        iteration = iteration + 1
        if iteration == counter:
            iteration = 0
            counter_columns = counter_columns + 1
            print("column :",counter_columns, "finished")
        if y >= pp_min and count == False:
            duration = duration + 1
            count = True
            start_period = index_list[iteration]
            event_amount = y
            index = iteration
            pp_sum = pp_sum + y
        elif y >= pp_min and count == True or y >= pp_max_1 and count == True:
            end_period = index_list[iteration]
            dry_periods = dry_periods.append({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount, "cell":columns[counter_columns]},ignore_index=True).sort_values('period_range',ascending=False)
            duration = 0
            count = False
            pp_sum = 0
        elif pp <= pp_max_1 and count == True:
            duration = duration + 1
            pp_sum = pp_sum + y
        else:
            continue
print(dry_periods.head(5))



In [1]:
import radproc as rp
from datetime import datetime, timedelta, date
import datetime
import pandas as pd
import time
import numpy as np

start = time.time()
#Existing Data
df = rp.hdf5_to_days(r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\aoi\out\aoi.h5", year_start=2014, year_end=2018)


periods=[]

for cell in df.columns:
    sub = pd.DataFrame({'amount': df[cell].values}, index=df.index)
    sub['flag'] = pd.cut(sub['amount'], [0.11, 15, np.inf],
                         labels=[0, 1]).astype(np.float)
    sub.loc[sub.flag>0, 'flag']=sub.loc[sub.flag>0, 'flag'].cumsum()
    sub.flag.ffill(inplace=True)
    x = sub[sub.flag>0].reset_index().groupby('flag').agg(
        {'Date (UTC)':['min', 'max'], 'amount': 'sum'})
    x.columns = ['start', 'end', 'amount']
    x['period_range'] = (x.end - x.start).dt.days + 1
    x['cell'] = cell
    x.reindex(columns=['start', 'end', 'period_range', 'cell'])
    periods.append(x)

resul = pd.concat(periods).reset_index(drop=True)

print(resul)

end = time.time()
print("processing time: ", end-start)

DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04',
               '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',
               '2014-01-09', '2014-01-10',
               ...
               '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25',
               '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
               '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns, UTC]', name=u'Date (UTC)', length=1826, freq='D')
DatetimeIndex(['2019-07-04 18:36:24.680000', '2019-07-05 18:36:24.680000',
               '2019-07-06 18:36:24.680000', '2019-07-07 18:36:24.680000',
               '2019-07-08 18:36:24.680000', '2019-07-09 18:36:24.680000',
               '2019-07-10 18:36:24.680000', '2019-07-11 18:36:24.680000',
               '2019-07-12 18:36:24.680000', '2019-07-13 18:36:24.680000'],
              dtype='datetime64[ns]', freq=None)
                           start                       end     amount  \
0      2014-07-08 00:00:00

In [5]:
#Export to csv file
resul.to_csv(r"C:\Users\USER\Desktop\Masterarbeit\DATA\RAD\analyzed\dry_periods.csv")

In [ ]:
#Create new DataFrame for exporting to GDB
#new_df = df.pivot(index=dailySum.index, columns='cell')
new_df = dailySum.iloc[0:0]
index_df = dailySum.index
bla = pd.concat([new_df,index_df],axis=0)
print(bla)

In [ ]:
from datetime import datetime, timedelta, date
import datetime
import pandas as pd
#print(dailySum.head(5))
#DataFrame 
dry_periods = pd.DataFrame(columns=("start_period","end_period","period_range","period_amount","event_amount"))
dry_periods.start_period = dry_periods.start_period.astype('datetime64[ns]')
dry_periods.end_period = dry_periods.end_period.astype('datetime64[ns]')
#dry_periods.period_range = dry_periods.period_range.astype('int64')
#dry_periods.period_amount = dry_periods.period_amount.astype('float64')
#dry_periods.event_amount = dry_periods.event_amount.astype('float64')

#print(dry_periods.dtypes)
#dry_periods = dry_periods(start_period, dtype='datetime64[ns]')
#print(dry_periods.head(5))
#new_entry = pd.DataFrame({"start_period":datetime.date.today() ,"end_period":datetime.date.today() + timedelta(days=1),"period_range": 10,"period_amount": 1,"event_amount":12},index=[1])
#dry_periods.update(new_entry)
#print(dry_periods.head(5))
columns = dailySum.columns
print(len(columns))
numbers = range(len(columns))
d = dict(zip(numbers,columns))
#print(d)
#dailySum.round(2)
#print(d)

#Variables 
index_list = dailySum.index #Index for updating df / Integear
#print(type(index_list),index_list)
#start_period = datetime.date.today() #Datetime object. Event Date of heavy rainfall
#end_period = datetime.date.today()  #Datetime object. End of dry period after Event
period_range = 0  #Amount of days after Event without much rain Integear
period_amount = 0 #Amount of PP in dry days except event Integear
event_amount = 0.0  #Amount of heavy rainfall on the event date Float
#print(dailySum)
#Conditions
iteration = -1 #Iterations Steps
duration = 0 #days
duration_min_bevor = 4 #min dry days bevore survey start #necessary????
duration_min_after = 10 #min dry days after survey
pp = 0 #current pp
pp_sum = 0.0 #mm
pp_min = 15.0 #mm min pp for start to count dry days until duration_min_after
pp_max_1 = 0.11 #max pp for 1 day while counting dry days
pp_max_2 = 0.5 #max pp for 1 week while counting dry days
dry_days = 0 #dry days after event
row = -1 #Panda rows
count = False

for x in dailySum[columns[0]]:
    iteration = iteration + 1
    if x >= pp_min and count == False:
        duration = duration + 1
        count = True
        row = row + 1
        start_period = index_list[iteration]
        event_amount = x
        index = iteration
        pp_sum = pp_sum + x
        continue
    elif x >= pp_min and count == True or x >= pp_max_1 and count == True:
        end_period = index_list[iteration]
        #new_entry = pd.DataFrame({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount} ,index=[1])
        #dry_periods.update(new_entry)
        dry_periods = dry_periods.append({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount},ignore_index=True)
        duration = 0
        count = False
        pp_sum = 0
    elif pp <= pp_max_1 and count == True:
        duration = duration + 1
        pp_sum = pp_sum + x
    else:
        continue

#print(dry_periods)
dictt = {}
def test():
    index_list = dailySum.index #Index for updating df / Integear
    period_range = 0  #Amount of days after Event without much rain Integear
    period_amount = 0 #Amount of PP in dry days except event Integear
    event_amount = 0.0  #Amount of heavy rainfall on the event date Float
    iteration = -1 #Iterations Steps
    duration = 0 #days
    duration_min_bevor = 4 #min dry days bevore survey start #necessary????
    duration_min_after = 10 #min dry days after survey
    pp = 0 #current pp
    pp_sum = 0.0 #mm
    pp_min = 15.0 #mm min pp for start to count dry days until duration_min_after
    pp_max_1 = 0.11 #max pp for 1 day while counting dry days
    pp_max_2 = 0.5 #max pp for 1 week while counting dry days
    dry_days = 0 #dry days after event
    row = -1 #Panda rows
    count = False
    dry_periods = pd.DataFrame(columns=("start_period","end_period","period_range","period_amount","event_amount","cell"))
    dry_periods.start_period = dry_periods.start_period.astype('datetime64[ns]')
    dry_periods.end_period = dry_periods.end_period.astype('datetime64[ns]')
    columns = dailySum.columns
    numbers = range(len(columns))
    d = dict(zip(numbers,columns))
    counter = 1826 #4 normal 1 schaltjahr (366)
    counter_columns = 0
    #dailySum1 = dailySum.iloc[0:0]
    for column in dailySum:
        for x in dailySum[columns]:
            iteration = iteration + 1
            print(iteration)
            if iteration == counter:
                iteration = 0
                counter_columns = counter_columns + 1
                print("Year :",counter_columns, "finished")
            if x >= pp_min and count == False:
                duration = duration + 1
                count = True
                row = row + 1
                start_period = index_list[iteration]
                event_amount = x
                index = iteration
                pp_sum = pp_sum + x
                continue
            elif x >= pp_min and count == True or x >= pp_max_1 and count == True:
                end_period = index_list[iteration]
                #new_entry = pd.DataFrame({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount} ,index=[1])
                #dry_periods.update(new_entry)
                dry_periods = dry_periods.append({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount, "cell":columns[counter_columns]},ignore_index=True).sort_values('period_range',ascending=False)
                duration = 0
                count = False
                pp_sum = 0
            elif pp <= pp_max_1 and count == True:
                duration = duration + 1
                pp_sum = pp_sum + x
            else:
                continue
         #return dictt.update({dry_periods.iat[0,4]:{"start_period":dry_periods["start_period"] ,"end_period":dry_periods["end_period"],"period_range":dry_periods["period_range"],"period_amount":dry_periods["period_amount"] ,"event_amount":dry_periods["event_amount"]}})
    return dry_periods #.to_dict(orient='dict')

finish = test()
print(finish.head(10))
print("finished")
#print(test())
#print("asdasda",test(1).iloc[0]["start_period"])
#dailySum1 = dailySum.iloc[0:0]    
#print(test(1).iloc[0:3,0],test(1).iloc[0:3,5])
#print(test(100).iloc[0:3,0])
#dailySum1.iat[0,0] = test(0).iloc[0:5]
#dailySum1.486335 = dry_periods.486335.astype('datetime64[ns]')
#dailySum1 = dailySum1.append(test(0).iloc[0]["start_period"])
#print(dailySum1)

#columns = dailySum1.columns
#for x in range(10):
#    dailySum1.iat[0,0] = test(x)
#print("asdasdad",dailySum1.iat[0,0])
#dailySum1.iat[0,0] = dry_periods.iat[0,0]
#raster1 = test(0)
#raster2 = test(1)
#print(dailySum1)
#df = pd.DataFrame(columns=df.columns)

In [ ]:
df = dailySum.head(10)
print((df.index))

for column in df:
    for row in df[column]:
        print(row)



In [ ]:
from datetime import datetime, timedelta, date
import datetime
import pandas as pd
#print(dailySum.head(5))
#DataFrame 
dry_periods = pd.DataFrame(columns=("start_period","end_period","period_range","period_amount","event_amount"))
dry_periods.start_period = dry_periods.start_period.astype('datetime64[ns]')
dry_periods.end_period = dry_periods.end_period.astype('datetime64[ns]')
#dry_periods.period_range = dry_periods.period_range.astype('int64')
#dry_periods.period_amount = dry_periods.period_amount.astype('float64')
#dry_periods.event_amount = dry_periods.event_amount.astype('float64')

#print(dry_periods.dtypes)
#dry_periods = dry_periods(start_period, dtype='datetime64[ns]')
#print(dry_periods.head(5))
#new_entry = pd.DataFrame({"start_period":datetime.date.today() ,"end_period":datetime.date.today() + timedelta(days=1),"period_range": 10,"period_amount": 1,"event_amount":12},index=[1])
#dry_periods.update(new_entry)
#print(dry_periods.head(5))
columns = dailySum.columns
print(len(columns))
numbers = range(len(columns))
d = dict(zip(numbers,columns))
#print(d)
#dailySum.round(2)
#print(d)

#Variables 
index_list = dailySum.index #Index for updating df / Integear
#print(type(index_list),index_list)
#start_period = datetime.date.today() #Datetime object. Event Date of heavy rainfall
#end_period = datetime.date.today()  #Datetime object. End of dry period after Event
period_range = 0  #Amount of days after Event without much rain Integear
period_amount = 0 #Amount of PP in dry days except event Integear
event_amount = 0.0  #Amount of heavy rainfall on the event date Float
#print(dailySum)
#Conditions
iteration = -1 #Iterations Steps
duration = 0 #days
duration_min_bevor = 4 #min dry days bevore survey start #necessary????
duration_min_after = 10 #min dry days after survey
pp = 0 #current pp
pp_sum = 0.0 #mm
pp_min = 15.0 #mm min pp for start to count dry days until duration_min_after
pp_max_1 = 0.11 #max pp for 1 day while counting dry days
pp_max_2 = 0.5 #max pp for 1 week while counting dry days
dry_days = 0 #dry days after event
row = -1 #Panda rows
count = False

index_list = dailySum.index #Index for updating df / Integear
period_range = 0  #Amount of days after Event without much rain Integear
period_amount = 0 #Amount of PP in dry days except event Integear
event_amount = 0.0  #Amount of heavy rainfall on the event date Float
iteration = -1 #Iterations Steps
duration = 0 #days
duration_min_bevor = 4 #min dry days bevore survey start #necessary????
duration_min_after = 10 #min dry days after survey
pp = 0 #current pp
pp_sum = 0.0 #mm
pp_min = 15.0 #mm min pp for start to count dry days until duration_min_after
pp_max_1 = 0.11 #max pp for 1 day while counting dry days
pp_max_2 = 0.5 #max pp for 1 week while counting dry days
dry_days = 0 #dry days after event
row = -1 #Panda rows
count = False
dry_periods = pd.DataFrame(columns=("start_period","end_period","period_range","period_amount","event_amount","cell"))
dry_periods.start_period = dry_periods.start_period.astype('datetime64[ns]')
dry_periods.end_period = dry_periods.end_period.astype('datetime64[ns]')
columns = dailySum.columns
numbers = range(len(columns))
d = dict(zip(numbers,columns))
         #dailySum1 = dailySum.iloc[0:0]
for y in range(10):
    for x in dailySum[columns[y]]:
        iteration = iteration + 1
        if x >= pp_min and count == False:
            duration = duration + 1
            count = True
            row = row + 1
            start_period = index_list[iteration]
            event_amount = x
            index = iteration
            pp_sum = pp_sum + x
            continue
        elif x >= pp_min and count == True or x >= pp_max_1 and count == True:
            end_period = index_list[iteration]
            #new_entry = pd.DataFrame({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount} ,index=[1])
            #dry_periods.update(new_entry)
            dry_periods = dry_periods.append({"start_period":start_period ,"end_period":end_period,"period_range":duration,"period_amount":pp_sum ,"event_amount":event_amount, "cell":columns[y]},ignore_index=True).sort_values('period_range',ascending=False)
            duration = 0
            count = False
            pp_sum = 0
        elif pp <= pp_max_1 and count == True:
            duration = duration + 1
            pp_sum = pp_sum + x
        else:
            continue
            
print(dry_periods)

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
#Source
df = dailySum

#Conditions
duration = 0 #days
duration_min_bevor = 4 #min dry days bevore survey start #necessary????
duration_min_after = 10 #min dry days after survey
pp = 0 #current pp
pp_sum = 0.0 #mm
pp_min = 15.0 #mm min pp for start to count dry days until duration_min_after
pp_max_1 = 0.1 #max pp for 1 day while counting dry days
pp_max_2 = 0.5 #max pp for 1 week while counting dry days
dry_days = 0 #dry days after event

#Targets
days = pd.DataFrame(columns=("event_date","dry_days","event_amount","amount_dry_period"))
print(days.head())

for x in df:
    pp_sum = pp_sum + x[mm]
    pp = x[mm]
    print("actual:", pp ,"sum:", pp_sum)
    if pp_sum > pp_min:
        duration = duration + 1
        print("duration:", duration)
        if duration == duration_min_after:
            event_date = df.date[x] - timedelta(days=duration_min_after)
            event_amount = df.amount[x.index-10] 
            index = df.index[x]
            print("event_date: ", event_date,"event_amount: ",event_amount, "index: ,", index)
            days = days.append([{"event_date":event_date,"event_amount":event_amount}], ignore_index=True) #date of event 
        elif duration > duration_min_after and pp <= pp_max_1:
            dry_days = dry_days + 1
        elif duration < duration_min_after:
            continue
        else:
            df.set_value(index=index, col="dry_days", value=dry_days, takeable=False)
            days.iloc[index] = days
            dry_days = 0
            duration = 0
            pp_sum = 0
            continue
    else:
        continue 
                                 
                                 
print(days.head())                                 
                
                
                

        
                                 
                                 
                                 
            
#d = datetime.today() - timedelta(days=days_to_subtract)

#res = pd.DataFrame(columns=('lib', 'qty1', 'qty2'))
#res = res.append([{'qty1':10.0}], ignore_index=True)
        